In [1]:
import pandas as pd

# Importing Dataset

In [2]:
df=pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df=df.drop('link',axis=1).reset_index(drop=True) #droping the colum & reseting the old index of colum

In [8]:
df.head(0)

,artist,song,text


In [9]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [10]:
df=df.sample(5000).reset_index(drop=True)  #sample song taking & reseting index

In [11]:
df.shape

(5000, 3)

# Text Cleaning / Text Preprocessing

In [12]:
df['text']=df['text'].str.lower().replace(r'^\w\s','').replace(r'\n',' ',regex=True)

In [13]:
df.tail(5)

,artist,song,text
4995,Chris Brown,Biggest Fan,you would think we in a waterbed \r all the s...
4996,The Temptations,I Wish You Love,"goodbye, no use leading with our chins \r thi..."
4997,Phil Collins,No Matter Who,"well it's a long, long road when you're out th..."
4998,Status Quo,Lies,i had another drink after another drink and th...
4999,Unseen,Where Have You Gone?,i've seen so many kids over the years \r a fe...


In [14]:
import nltk
from nltk.stem.porter import PorterStemmer

In [15]:
stemmer=PorterStemmer()

In [16]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [17]:
token("you are")

'you are'

In [18]:
df['text'].apply(lambda x:token(x))

0       daybreak find you up and aliv just as though y...
1       they 're tryin ' to take away my pride by stri...
2       wild wine - befor your castl wild wine - i am ...
3       i put my fear back in it box and i put the box...
4       light to men love of god heal for the wound he...
                              ...                        
4995    you would think we in a waterb all the sheet a...
4996    goodby , no use lead with our chin thi is wher...
4997    well it 's a long , long road when you 're out...
4998    i had anoth drink after anoth drink and then i...
4999    i 've seen so mani kid over the year a few yea...
Name: text, Length: 5000, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [21]:
matrix=tfid.fit_transform(df['text'])

In [22]:
similar=cosine_similarity(matrix) #creating similar % like tf

In [23]:
similar[0]

array([1.        , 0.03212475, 0.0160054 , ..., 0.14318178, 0.05511379,
       0.05508767])

In [24]:
no=df[df['song']=='I Wish You Love'].index[0]
no

512

In [25]:
def recommender(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:11]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [26]:
recommender("I Wish You Love")

['I Wish You Love',
 'I Wish I Knew How It Would Feel to Be Free',
 'All The Best',
 'I Wish',
 'Make A Wish',
 'I Will Always Love You',
 'Sometimes',
 "It's So Cool",
 'Miss America',
 'Stuck In The Moment']

In [27]:
import pickle

In [28]:
pickle.dump(similar,open("similarity.pkl","wb"))

In [29]:
pickle.dump(df,open("df.pkl","wb"))